# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import psycopg2
from sql_queries import airport_insert, demographic_insert, immigration_insert, temperature_insert

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>
- US I94 Immigration data including demographics and temperature data will be used in this project

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 
- Data is provided by US National Tourism and Trade Office. Sample data is provided in csv format. 

## I94 Immigration Data

In [32]:
# Read in the data here
file = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
df_immigration = pd.read_sas(file, 'sas7bdat', encoding="ISO-8859-1")

In [33]:
df_immigration.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [34]:
df_immigration.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [35]:
df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [18]:
for index, row in df_immigration.iterrows():
    for value in row.values:
        if '\x00' in str(value):
            print(index)
            print(value)

658953
  
663650
  
704177
  
710658
  
717777
  
732872
  
732873
  
732874
  
936911
  
940514
  
940515
  
1071897
  
1071898
  
1091173
  
1095503
  
1095509
  
1102668
  
1103798
  
1103799
  
1105943
  
1122531
  
1122931
  
1141996
  
1146645
  
1146646
  
1146647
  
1146648
  
1146649
  
1146650
  
1146651
  
1146652
  
1146653
  
1152769
  
1153073
  
1153404
  
1162145
  
1162146
  
1162147
  
1162148
  
1163604
  
1163875
  
1164097
  
1164098
  
1169237
  
1169238
  
1169239
  
1169240
  
1169241
  
1169242
  
1169243
  
1169244
  
1174882
  
1175685
  
1177983
  
1178116
  
1178324
  
1178325
  
1178326
  
1178327
  
1178750
  
1178751
  
1178752
  
1178753
  
1184338
  
1187167
  
1187298
  
1187299
  
1187300
  
1193272
  
1194839
  
1194866
  
1194867
  
1194868
  
1195848
  
1197177
  
1206699
  
1216770
  
1232074
  
1232075
  
1239872
  
1245105
  
1245106
  
1245107
  
1247155
  
1247156
  
1255581
  
1259784
  
1259785
  
1259786
  
1259787
  
1263770
  
1263771
  

## US City Demographic Data
- Provided data from OpenSoft

In [36]:
df_demo = pd.read_csv("us-cities-demographics.csv", delimiter=";")

In [37]:
df_demo.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


## Airport Code Table
- data sample of airport codes and cities

In [38]:
df_air = pd.read_csv("airport-codes_csv.csv")

In [39]:
df_air.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


## World Temperature Data

In [40]:
filename = '../../data2/GlobalLandTemperaturesByCity.csv'
df_temp = pd.read_csv(filename)

In [41]:
df_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [42]:
df_us_temp = df_temp[df_temp['Country'] == "United States"]
df_us_temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
47555,1820-01-01,2.101,3.217,Abilene,United States,32.95N,100.53W
47556,1820-02-01,6.926,2.853,Abilene,United States,32.95N,100.53W
47557,1820-03-01,10.767,2.395,Abilene,United States,32.95N,100.53W
47558,1820-04-01,17.989,2.202,Abilene,United States,32.95N,100.53W
47559,1820-05-01,21.809,2.036,Abilene,United States,32.95N,100.53W


In [43]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [ ]:
#write to parquet
df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

In [44]:
df_immigration.describe()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94mode,depdate,i94bir,i94visa,count,biryear,admnum
count,3.096313e+06,3096313.0,3096313.0,3.096313e+06,3.096313e+06,3.096313e+06,3.096074e+06,2.953856e+06,3.095511e+06,3.096313e+06,3096313.0,3.095511e+06,3.096313e+06
mean,3.078652e+06,2016.0,4.0,3.049069e+02,3.032838e+02,2.055985e+04,1.073690e+00,2.057395e+04,4.176761e+01,1.845393e+00,1.0,1.974232e+03,7.082885e+10
std,1.763278e+06,0.0,0.0,2.100269e+02,2.085832e+02,8.777339e+00,5.158963e-01,2.935697e+01,1.742026e+01,3.983910e-01,0.0,1.742026e+01,2.215442e+10
min,6.000000e+00,2016.0,4.0,1.010000e+02,1.010000e+02,2.054500e+04,1.000000e+00,1.517600e+04,-3.000000e+00,1.000000e+00,1.0,1.902000e+03,0.000000e+00
25%,1.577790e+06,2016.0,4.0,1.350000e+02,1.310000e+02,2.055200e+04,1.000000e+00,2.056100e+04,3.000000e+01,2.000000e+00,1.0,1.962000e+03,5.603523e+10
50%,3.103507e+06,2016.0,4.0,2.130000e+02,2.130000e+02,2.056000e+04,1.000000e+00,2.057000e+04,4.100000e+01,2.000000e+00,1.0,1.975000e+03,5.936094e+10
75%,4.654341e+06,2016.0,4.0,5.120000e+02,5.040000e+02,2.056700e+04,1.000000e+00,2.057900e+04,5.400000e+01,2.000000e+00,1.0,1.986000e+03,9.350987e+10
max,6.102785e+06,2016.0,4.0,9.990000e+02,7.600000e+02,2.057400e+04,9.000000e+00,4.542700e+04,1.140000e+02,3.000000e+00,1.0,2.019000e+03,9.991557e+10


In [45]:
df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3096313 entries, 0 to 3096312
Data columns (total 28 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
visapost    object
occup       object
entdepa     object
entdepd     object
entdepu     object
matflag     object
biryear     float64
dtaddto     object
gender      object
insnum      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(15)
memory usage: 661.4+ MB


In [46]:
df_demo.describe()

,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,Count
count,2891.000000,2.888000e+03,2.888000e+03,2.891000e+03,2878.000000,2.878000e+03,2875.000000,2.891000e+03
mean,35.494881,9.732843e+04,1.017696e+05,1.989668e+05,9367.832523,4.065360e+04,2.742543,4.896377e+04
std,4.401617,2.162999e+05,2.315646e+05,4.475559e+05,13211.219924,1.557491e+05,0.433291,1.443856e+05
min,22.900000,2.928100e+04,2.734800e+04,6.321500e+04,416.000000,8.610000e+02,2.000000,9.800000e+01
25%,32.800000,3.928900e+04,4.122700e+04,8.042900e+04,3739.000000,9.224000e+03,2.430000,3.435000e+03
50%,35.300000,5.234100e+04,5.380900e+04,1.067820e+05,5397.000000,1.882200e+04,2.650000,1.378000e+04
75%,38.000000,8.664175e+04,8.960400e+04,1.752320e+05,9368.000000,3.397175e+04,2.950000,5.444700e+04
max,70.500000,4.081698e+06,4.468707e+06,8.550405e+06,156961.000000,3.212500e+06,4.980000,3.835726e+06


In [47]:
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [48]:
df_air.describe()

,elevation_ft
count,48069.000000
mean,1240.789677
std,1602.363459
min,-1266.000000
25%,205.000000
50%,718.000000
75%,1497.000000
max,22000.000000


In [49]:
df_air.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.0+ MB


In [50]:
df_temp.describe()

,AverageTemperature,AverageTemperatureUncertainty
count,8.235082e+06,8.235082e+06
mean,1.672743e+01,1.028575e+00
std,1.035344e+01,1.129733e+00
min,-4.270400e+01,3.400000e-02
25%,1.029900e+01,3.370000e-01
50%,1.883100e+01,5.910000e-01
75%,2.521000e+01,1.349000e+00
max,3.965100e+01,1.539600e+01


In [51]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


#### Cleaning Steps
Document steps necessary to clean the data

In [52]:
# Get port locations
with open("./I94_SAS_Labels_Descriptions.SAS") as f:
    content = f.readlines()
content = [x.strip() for x in content]
ports = content[302:962]
splitted_ports = [port.split("=") for port in ports]
port_codes = [x[0].replace("'","").strip() for x in splitted_ports]
port_locations = [x[1].replace("'","").strip() for x in splitted_ports]
port_cities = [x.split(",")[0] for x in port_locations]
port_states = [x.split(",")[-1] for x in port_locations]
df_port_locations = pd.DataFrame({"port_code" : port_codes, "port_city": port_cities, "port_state": port_states})
df_port_locations.head(20)

,port_code,port_city,port_state
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK
5,DTH,DUTCH HARBOR,AK
6,EGL,EAGLE,AK
7,FRB,FAIRBANKS,AK
8,HOM,HOMER,AK
9,HYD,HYDER,AK


In [53]:
df_disregard_ports = df_port_locations[df_port_locations["port_city"] == df_port_locations["port_state"]]
disregard_ports = list(set(df_disregard_ports["port_code"].values))
print(disregard_ports)

['PLB', 'NK', 'MAA', 'EGE', 'GPI', 'A2A', 'YGF', 'DRV', 'GAC', 'SCH', 'SP0', '5T6', 'JMZ', 'XXX', 'PFN', 'JBQ', 'IAG', 'STN', 'JFA', 'WTR', 'W55', 'XNA', 'SUS', 'CHN', '74S', 'MTH', 'AG', 'TIW', 'NYL', 'FTB', 'MAP', 'X44', 'VMB', 'BCM', 'PHN', 'X96', 'RYY', 'NGL', 'ASI', 'T01', 'JSJ', 'Y62', 'OSN', 'UNK', '888', 'OLM', 'AKT', 'GMT', 'ISP', 'OGS', 'CXO', 'AUH', 'WAS', 'FRG', 'NC8', 'WA5', 'JIG', 'CP', 'PHF', 'ATW', 'OTS', 'OAI', 'CPX', 'AMT', 'LIT', 'FSC', 'BKF', 'DEC', 'PCW', 'DAY', '.GA', 'ADU', 'HRL', '060', 'CLX', 'BUS', 'ZZZ']


In [54]:
df_immigration = df_immigration[~df_immigration["i94port"].isin(disregard_ports)]

In [55]:
# Performing cleaning tasks here
df_immigration.drop(columns=['insnum', 'entdepu', 'occup', 'visapost'], inplace=True)
df_immigration.dropna(inplace=True)

In [56]:
df_immigration.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2306754 entries, 12 to 3029508
Data columns (total 24 columns):
cicid       float64
i94yr       float64
i94mon      float64
i94cit      float64
i94res      float64
i94port     object
arrdate     float64
i94mode     float64
i94addr     object
depdate     float64
i94bir      float64
i94visa     float64
count       float64
dtadfile    object
entdepa     object
entdepd     object
matflag     object
biryear     float64
dtaddto     object
gender      object
airline     object
admnum      float64
fltno       object
visatype    object
dtypes: float64(13), object(11)
memory usage: 440.0+ MB


In [73]:
df_immigration = df_immigration[df_immigration.i94addr != '\x00\x00']

In [74]:
df_immigration[df_immigration.i94addr == '\x00\x00']

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepa,entdepd,matflag,biryear,dtaddto,gender,airline,admnum,fltno,visatype


In [75]:
df_us_temp.dropna(inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [76]:
df_air.dropna(subset=['iata_code'], inplace=True)

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model


Immigrants Table holds the core information needed and dimension tables (temperature, demographics, airport) can be joined if additional information is  neccessary 

| Table |Description |
| --- |-----------|
| immigrants | Fact table for i94 immigrations data|
| temperature | Dimension table for temperature data|
| demographics| Dimension table for city demographics data|
| airport | Dimension table for airports information data|

## Immigrants

| Column |Type|
| --- |-----------|
|cicid  |  FLOAT PRIMARY KEY|
|    year  |   FLOAT|
|    month  |  FLOAT|
|    cit    |  FLOAT|
|    res    |  FLOAT|
|    iata   |  VARCHAR(3)|
|    arrdate | FLOAT|
|    mode   |  FLOAT|
|    addr   |  VARCHAR|
|    depdate | FLOAT|
|    bir     | FLOAT|
|    visa    | FLOAT|
|    count   | FLOAT|
|    dtadfile |VARCHAR|
|    entdepa | VARCHAR(1)|
|    entdepd | VARCHAR(1)|
|    matflag | VARCHAR(1)|
|    biryear | FLOAT|
|    dtaddto | VARCHAR|
|    gender  | VARCHAR(1)|
|    airline | VARCHAR|
|    admnum  | FLOAT |
|    fltno   | VARCHAR |
|   visatype | VARCHAR |

## Temperature

| Column |Type|
| --- |-----------|
|    timestamp              |        DATE|
|    average_temperature     |       FLOAT|
|    average_temperature_uncertainty | FLOAT|
|    city                      |     VARCHAR|
|    country                   |     VARCHAR|
|    latitude                   |    VARCHAR|
|    longitude                 |     VARCHAR|

## Demographics

| Column |Type|
| --- |-----------|
|    city           |        VARCHAR|
|    state          |        VARCHAR|
|    media_age      |        FLOAT|
|    male_population |       INT|
|    female_population  |    INT|
|    total_population   |    INT|
|    num_veterans       |    INT|
|    foreign_born       |    INT|
|    average_household_size | FLOAT|
|    state_code         |    VARCHAR(2)|
|    race               |    VARCHAR|
|    count              |    INT|

## Airports

| Column |Type|
| --- |-----------|
|    iata_code  |  VARCHAR PRIMARY KEY |
|    name       |  VARCHAR |
|    type       |  VARCHAR |
|    local_code  | VARCHAR |
|    coordinates | VARCHAR|
|    city        | VARCHAR|
|    elevation_ft | FLOAT|
|    continent  |  VARCHAR|
|    iso_country | VARCHAR|
|    iso_region  | VARCHAR|
|    municipality  | VARCHAR|
|    gps_code   |  VARCHAR|

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model
- execute create_tables.py to create tables
- create airports data
- insert data

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [34]:
#cur.close()

In [77]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [78]:
df_air= df_air.merge(df_port_locations, left_on="iata_code", right_on="port_code")
df_air.drop_duplicates(subset=['iata_code'], inplace=True)
df_air.drop(columns=["port_code"], inplace=True)
df_air = df_air[["iata_code", "name", "type", "local_code", "coordinates", "port_city", "elevation_ft", "continent", "iso_country", "iso_region", "municipality", "gps_code"]]

for index, row in df_air.iterrows():
    cur.execute(airport_insert, list(row.values))
    conn.commit()

In [ ]:
for index, row in df_immigration.iterrows():
    cur.execute(immigration_insert, list(row.values))
    conn.commit()

In [79]:
for index, row in df_demo.iterrows():
    cur.execute(demographic_insert, list(row.values))
    conn.commit()

In [ ]:
for index, row in df_temp.iterrows():
    cur.execute(temperature_insert, list(row.values))
    conn.commit()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here
tables = ['immigrations', 'airports', 'demographics', 'temperature']

for table in tables:
    cur.execute(f"SELECT COUNT(*) FROM {table}")
    conn.commit()
    if cur.rowcount < 1:
        print(f'empty data in {table} table')
    


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
     - Data should be processed in a distributed way, hence use Spark with EMR
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
     - Schedule a Dag using Airflow at the desired time and have it run daily.
 * The database needed to be accessed by 100+ people.
     - Use AWS Reshift to easily process data accessed by many people and use db role to assign appropriate roles